In [1]:
# mkdir data
# cd data
# wget https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/vehicles.zip
# unzip vehicles.zip
# wget https://s3.amazonaws.com/udacity-sdc/Vehicle_Tracking/non-vehicles.zip
# unzip non-vehicles.zip
# https://github.com/udacity/self-driving-car/tree/master/annotations
# wget http://bit.ly/udacity-annotations-autti

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import sklearn

from glob import glob
#from sklearn.cross_validation import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from skimage.feature import hog
from time import time

#np.set_printoptions(threshold=np.nan)
np.set_printoptions(threshold=10)

random.seed(0)

%matplotlib inline

In [3]:
PREFIX_PATH = '/Users/tringuyen/OneDrive/Documents/2017 - Udacity Self Driving Car course/CarND-Vehicle-Detection/'
# PREFIX_PATH = '/var/storage/shared/ipgsrch/trnguye/Google-Neural-Machine-Translation-GNMT-/tmp/CarND-Vehicle-Detection/'

PATH_TO_GTI_FAR_VEHICLES = \
    PREFIX_PATH + 'data/vehicles/GTI_Far/*.png'
PATH_TO_GTI_LEFT_VEHICLES = \
    PREFIX_PATH + 'data/vehicles/GTI_Left/*.png'
PATH_TO_GTI_CLOSE_VEHICLES = \
    PREFIX_PATH + 'data/vehicles/GTI_MiddleClose/*.png'
PATH_TO_GTI_RIGHT_VEHICLES = \
    PREFIX_PATH + 'data/vehicles/GTI_Right/*.png'
PATH_TO_KITTI_VEHICLES = \
    PREFIX_PATH + 'data/vehicles/KITTI*/*.png'
PATH_TO_GTI_NON_VEHICLES = \
    PREFIX_PATH + 'data/non-vehicles/GTI/*.png'
PATH_TO_KITTI_NON_VEHICLES = \
    PREFIX_PATH + 'data/non-vehicles/Extras/*.png'
    
PATH_TO_TEST_IMAGES = PREFIX_PATH + 'test_images/*.jpg'

PATH_TO_TEST_VIDEO_INTPUT = PREFIX_PATH + 'test_video.mp4'
PATH_TO_TEST_VIDEO_OUTPUT = PREFIX_PATH + 'test_video_out.mp4'

PATH_TO_PROJECT_VIDEO_INTPUT = PREFIX_PATH + 'project_video.mp4'
PATH_TO_PROJECT_VIDEO_OUTPUT = PREFIX_PATH + 'project_video_out.mp4'

In [4]:
HEATMAP_THRESHOLD = 4
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

In [5]:
MANUAL_SPLIT_VEHICLE_FILE_PATHS = [PATH_TO_GTI_FAR_VEHICLES, 
                                   PATH_TO_GTI_LEFT_VEHICLES, 
                                   PATH_TO_GTI_CLOSE_VEHICLES, 
                                   PATH_TO_GTI_RIGHT_VEHICLES]
RANDOM_SPLIT_VEHICLE_FILE_PATHS = [PATH_TO_KITTI_VEHICLES]
MANUAL_SPLIT_NON_VEHICLE_FILE_PATHS = [PATH_TO_GTI_NON_VEHICLES]
RANDOM_SPLIT_NON_VEHICLE_FILE_PATHS = [PATH_TO_KITTI_NON_VEHICLES]
TRAIN_DEV_TEST_SPLIT = (0.8, 0.1, 0.1)

In [6]:
def load_data(p_manual_split_file_paths, p_random_split_file_paths):
    output_train = []
    output_dev = []
    output_test = []

    if p_manual_split_file_paths is not None:
        for paths_to_file in p_manual_split_file_paths:
            paths_to_file = glob(paths_to_file)
            nb_files = len(paths_to_file)
            train, dev, test = \
                np.split(paths_to_file, 
                         [int(nb_files * TRAIN_DEV_TEST_SPLIT[0]), 
                          int(nb_files * (TRAIN_DEV_TEST_SPLIT[0] + TRAIN_DEV_TEST_SPLIT[1]))])
            output_train.append(train)
            output_dev.append(dev)
            output_test.append(test)
                
    for paths_to_file in p_random_split_file_paths:
        paths_to_file = glob(paths_to_file)
        train, dev_test = train_test_split(paths_to_file, train_size=TRAIN_DEV_TEST_SPLIT[0])
        dev, test = \
            train_test_split(
                dev_test,
                train_size=TRAIN_DEV_TEST_SPLIT[1] / (TRAIN_DEV_TEST_SPLIT[1] + TRAIN_DEV_TEST_SPLIT[2]))
        output_train.append(train)
        output_dev.append(dev)
        output_test.append(test)

    return np.concatenate(output_train), np.concatenate(output_dev), np.concatenate(output_test) 

In [7]:
def shuffle_image_paths(p_vehicle_image_file_paths, p_non_vehicle_image_file_paths):
    image_paths = np.concatenate((p_vehicle_image_file_paths, p_non_vehicle_image_file_paths))
    labels = np.concatenate((np.ones(len(p_vehicle_image_file_paths)), 
                             np.zeros(len(p_non_vehicle_image_file_paths))))
    image_paths, labels = shuffle(image_paths, labels)
    
    return image_paths, labels

In [8]:
def extract_features_from_single_image(
        p_image_file_path, 
        p_color_space='YCrCb', 
        p_resize=(64, 64),
        p_color_histogram_bin=32,
        p_color_histogram_bin_range=(0, 256),
        p_hog_orient=9, 
        p_hog_pix_per_cell=8, 
        p_hog_cell_per_block=2,
        p_feature_types=['bin spacial', 'histogram', 'hog']):

    image = cv2.imread(p_image_file_path)

    if p_color_space == 'HSV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif p_color_space == 'LUV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif p_color_space == 'HLS':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    elif p_color_space == 'YUV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    elif p_color_space == 'YCrCb':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)     

    image = cv2.resize(image, p_resize)
        
    features = np.array([])
    if 'bin spacial' in p_feature_types:
        bin_spatial_features = image.ravel()
        features = np.concatenate((features, bin_spatial_features))
    
    if 'histogram' in p_feature_types:
        color_histograms = []
        for channel in range(image.shape[2]):
            color_histograms.append(np.histogram(image[:, :, channel], 
                                                 bins=p_color_histogram_bin, 
                                                 range=p_color_histogram_bin_range)[0])
        features = np.concatenate((features, np.ravel(color_histograms)))

    if 'hog' in p_feature_types:
        hog_features = []
        for channel in range(image.shape[2]):
            hog_features.append(
                hog(image[:, :, channel],
                    orientations=p_hog_orient,
                    pixels_per_cell=(p_hog_pix_per_cell, p_hog_pix_per_cell),
                    cells_per_block=(p_hog_cell_per_block, p_hog_cell_per_block),
                    transform_sqrt=True,
                    visualise=False))
        features = np.concatenate((features, np.ravel(hog_features)))

    return features

In [9]:
def extract_features_from_file_paths(p_image_file_paths):
    features = []
    
    for image_file_path in p_image_file_paths:
        features.append(extract_features_from_single_image(image_file_path))

    features = np.vstack(features)
    
    return features

In [10]:
def train_classifier(p_verbal=True):            
    train_vehicles_file_paths, dev_vehicles_file_paths, test_vehicles_file_paths = \
        load_data(p_manual_split_file_paths=MANUAL_SPLIT_VEHICLE_FILE_PATHS, 
                  p_random_split_file_paths=RANDOM_SPLIT_VEHICLE_FILE_PATHS)
    train_non_vehicles_file_paths, dev_non_vehicles_file_paths, test_non_vehicles_file_paths = \
        load_data(p_manual_split_file_paths=MANUAL_SPLIT_NON_VEHICLE_FILE_PATHS, 
                  p_random_split_file_paths=RANDOM_SPLIT_NON_VEHICLE_FILE_PATHS)
        
    train_image_file_paths, train_labels = \
        shuffle_image_paths(train_vehicles_file_paths, train_non_vehicles_file_paths)
    dev_image_file_paths, dev_labels = \
        shuffle_image_paths(dev_vehicles_file_paths, dev_non_vehicles_file_paths)
    test_image_file_paths, test_labels = \
        shuffle_image_paths(test_vehicles_file_paths, test_non_vehicles_file_paths)
    
    if p_verbal:
        start_time = time()
    train_features = extract_features_from_file_paths(train_image_file_paths)
    dev_features = extract_features_from_file_paths(dev_image_file_paths)
    test_features = extract_features_from_file_paths(test_image_file_paths)
    
    all_features = np.concatenate((train_features, dev_features, test_features)).astype(np.float64)                        
    feature_scaler = StandardScaler().fit(all_features)
    train_scaled_features = feature_scaler.transform(train_features)
    dev_scaled_features = feature_scaler.transform(dev_features)
    test_scaled_features = feature_scaler.transform(test_features)
    if p_verbal:
        print('Extract features in %d seconds' % (time() - start_time))
    
    if p_verbal:
        start_time = time()
    # classifier = SVC()
    classifier = LinearSVC()
    # n_estimators = 10
    # classifier = BaggingClassifier(SVC(), max_samples=1.0 / n_estimators, n_estimators=n_estimators))

    # classifier = RandomForestClassifier(min_samples_leaf=20)
    classifier.fit(train_scaled_features, train_labels)
    if p_verbal:
        print('Train classifier in %d seconds' % (time() - start_time))

    if p_verbal:
        start_time = time()
        train_accuracy = classifier.score(train_scaled_features, train_labels)
        print()
        print('Train accuracy: %f' % train_accuracy)
        print('Evaluate train accuracy in %d seconds' % (time() - start_time))

        start_time = time()
        dev_accuracy = classifier.score(dev_scaled_features, dev_labels)
        print()
        print('Dev accuracy: %f' % dev_accuracy)
        print('Evaluate dev accuracy in %d seconds' % (time() - start_time))

        start_time = time()
        test_accuracy = classifier.score(test_scaled_features, test_labels)
        print()
        print('Test accuracy: %f' % test_accuracy)
        print('Evaluate test accuracy in %d seconds' % (time() - start_time))
        
        print()
        print('Sample predictions:')
        for i in range(10):
            random_num = random.randint(1, 10)
            image_file_path = test_image_file_paths[random_num]
            features = test_scaled_features[random_num].reshape(1, -1)
            label = test_labels[random_num]
            plt.figure()
            plt.imshow(cv2.imread(image_file_path)[:,:,::-1])
            plt.title('Label: %d ; Predict: %d' % (label, classifier.predict(features)))      
    
    return feature_scaler, classifier

In [11]:
feature_scaler, classifier = train_classifier(p_verbal=False)

In [12]:
def create_single_heat_map(
        p_image, 
        p_row_span,
        p_feature_scaler,
        p_classifier,
        p_image_window_size=(64, 64),
        p_classifier_color_space='YCrCb',
        p_classifier_resize=(64, 64),
        p_classifier_color_histogram_bin=32,
        p_classifier_color_histogram_bin_range=(0, 256),
        p_classifier_hog_orient=9, 
        p_classifier_hog_pix_per_cell=8, 
        p_classifier_hog_cell_per_block=2,
        p_classifier_feature_types=['bin spacial', 'histogram', 'hog']):
    
    heatmap = np.zeros((p_image.shape[0], p_image.shape[1]))

    search_area = p_image[p_row_span[0] : p_row_span[1], :, :]
    
    if p_classifier_color_space == 'HSV':
        search_area = cv2.cvtColor(search_area, cv2.COLOR_BGR2HSV)
    elif p_classifier_color_space == 'LUV':
        search_area = cv2.cvtColor(search_area, cv2.COLOR_BGR2LUV)
    elif p_classifier_color_space == 'HLS':
        search_area = cv2.cvtColor(search_area, cv2.COLOR_BGR2HLS)
    elif p_classifier_color_space == 'YUV':
        search_area = cv2.cvtColor(search_area, cv2.COLOR_BGR2YUV)
    elif p_classifier_color_space == 'YCrCb':
        search_area = cv2.cvtColor(search_area, cv2.COLOR_BGR2YCrCb) 
        
    row_scaler = p_image_window_size[0] / p_classifier_resize[0] 
    col_scaler = p_image_window_size[1] / p_classifier_resize[1]
    search_area = cv2.resize(search_area, 
                             (np.int(search_area.shape[1] / col_scaler), 
                              np.int(search_area.shape[0] / row_scaler)))

    n_row_blocks = (search_area.shape[0] // p_classifier_hog_pix_per_cell) - p_classifier_hog_cell_per_block + 1
    n_col_blocks = (search_area.shape[1] // p_classifier_hog_pix_per_cell) - p_classifier_hog_cell_per_block + 1

    n_row_blocks_per_window = \
        (p_classifier_resize[0] // p_classifier_hog_pix_per_cell) - p_classifier_hog_cell_per_block + 1
    n_col_blocks_per_window = \
        (p_classifier_resize[1] // p_classifier_hog_pix_per_cell) - p_classifier_hog_cell_per_block + 1

    cells_per_step = p_classifier_hog_cell_per_block // 2

    n_row_block_steps = (n_row_blocks - n_row_blocks_per_window) // cells_per_step
    n_col_block_steps = (n_col_blocks - n_col_blocks_per_window) // cells_per_step

    hog_image_features = []
    for channel in range(search_area.shape[2]):
        hog_image_features.append(
            hog(search_area[:, :, channel],
                orientations=p_classifier_hog_orient,
                pixels_per_cell=(p_classifier_hog_pix_per_cell, p_classifier_hog_pix_per_cell),
                cells_per_block=(p_classifier_hog_cell_per_block, p_classifier_hog_cell_per_block),
                transform_sqrt=True,
                visualise=False,
                feature_vector=False))

    for row_block_step in range(n_row_block_steps):
        for col_block_step in range(n_col_block_steps):

            row_block = row_block_step * cells_per_step
            col_block = col_block_step * cells_per_step

            hog_window_features = []
            for channel in range(search_area.shape[2]):
                hog_window_features.append(
                    hog_image_features[channel][
                        row_block : row_block + n_row_blocks_per_window, 
                        col_block : col_block + n_col_blocks_per_window])
                       
            hog_window_features = np.ravel(hog_window_features)

            row_top = row_block * p_classifier_hog_pix_per_cell
            col_left = col_block * p_classifier_hog_pix_per_cell

            window_image = search_area[row_top : row_top + p_classifier_resize[0], 
                                       col_left : col_left + p_classifier_resize[1]]
          
            bin_spatial_features = window_image.ravel()
                       
            color_histograms = []
            for channel in range(window_image.shape[2]):
                color_histograms.append(np.histogram(window_image[:, :, channel], 
                                                     bins=p_classifier_color_histogram_bin, 
                                                     range=p_classifier_color_histogram_bin_range)[0])
            color_histograms = np.ravel(color_histograms)

            features = np.concatenate((bin_spatial_features, color_histograms, hog_window_features))
            features = features.reshape(1, -1)
            features = p_feature_scaler.transform(features)
                       
            prediction = p_classifier.predict(features)
            if prediction == 1:
                heatmap[p_row_span[0] + int(row_top * row_scaler) : 
                            p_row_span[0] + int((row_top + p_classifier_resize[0]) * row_scaler), 
                        int(col_left * col_scaler) : 
                            int((col_left + p_classifier_resize[1]) * col_scaler)] += 1
            
    return heatmap

In [13]:
def create_aggregated_heat_map(
        p_image,
        p_row_span,
        p_feature_scaler,
        p_classifier,
        p_image_window_sizes=IMAGE_WINDOW_SIZES,
        p_verbose=True):
    
    heatmaps = []
    
    for image_window_size in p_image_window_sizes:
        heatmaps.append(create_single_heat_map(
            p_image=p_image,
            p_row_span=p_row_span,
            p_feature_scaler=p_feature_scaler,
            p_classifier=p_classifier,
            p_image_window_size=image_window_size))
    
    heatmaps = np.sum(heatmaps, axis=0)

    if p_verbose:
        plt.figure()
        plt.imshow(test_image[:,:,::-1])

        plt.figure()
        plt.imshow(heatmaps, cmap='hot', interpolation='nearest')
    
    return heatmaps

In [14]:
def draw_boxes(p_image, p_heatmap, p_threshold=HEATMAP_THRESHOLD, p_verbose=True):
    threshold, thresholded_image = cv2.threshold(
        p_heatmap.astype(np.uint8), thresh=p_threshold, maxval=1, type=cv2.THRESH_BINARY)
    
    im3, contours, hierarchy = cv2.findContours(thresholded_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for i in range(0, len(contours)):
       contour = contours[i]
       x, y, w, h = cv2.boundingRect(contour)
       cv2.rectangle(p_image, (x, y), (x + w, y + h), (0, 0, 255), 10)
    
    if p_verbose:
        plt.figure()
        plt.imshow(p_image[:, :, : : -1])
    
    return p_image, thresholded_image

In [15]:
"""
path_to_test_images = glob(PATH_TO_TEST_IMAGES)

test_image = cv2.imread(path_to_test_images[4])

heatmap = create_aggregated_heat_map(
    p_image=test_image,
    p_row_span=(400, 656),
    p_feature_scaler=feature_scaler,
    p_classifier=classifier)

draw_boxes(test_image, heatmap)
"""

'\npath_to_test_images = glob(PATH_TO_TEST_IMAGES)\n\ntest_image = cv2.imread(path_to_test_images[4])\n\nheatmap = create_aggregated_heat_map(\n    p_image=test_image,\n    p_row_span=(400, 656),\n    p_feature_scaler=feature_scaler,\n    p_classifier=classifier)\n\ndraw_boxes(test_image, heatmap)\n'

In [30]:
def process_video(
        p_path_to_input_video, 
        p_path_to_output_video,
        p_hitmap_threshold=HEATMAP_THRESHOLD,
        p_image_window_sizes=IMAGE_WINDOW_SIZES,
        p_sample=5):

    input_video = cv2.VideoCapture(p_path_to_input_video)
    output_video = None

    frame_num = -1
    while True:
        ret, frame = input_video.read()
        frame_num += 1
        if p_sample > 0 and (frame_num % p_sample) != 0:
            continue
            
        if ret == False:
            break

        if output_video is None:
            output_video = cv2.VideoWriter(
                p_path_to_output_video, 
                cv2.VideoWriter_fourcc(*'mp4v'), 
                30, 
                (frame.shape[1], int(frame.shape[0] * 1.5)))

        heatmap = create_aggregated_heat_map(
            p_image=frame,
            p_row_span=(400, 656),
            p_feature_scaler=feature_scaler,
            p_classifier=classifier,
            p_image_window_sizes=p_image_window_sizes,
            p_verbose=False)

        lower_output_frame, thresholded_frame = draw_boxes(frame, heatmap, p_threshold=p_hitmap_threshold, p_verbose=False)
        
        #heatmap = ny.ndarray(plt.imshow(heatmap, cmap='hot'))
        heatmap = heatmap / heatmap.max()
        cmap = plt.get_cmap('hot')
        rgba_img = cmap(heatmap)
        rgb_img = np.delete(rgba_img, 3, 2)
        heatmap = cv2.resize(rgb_img[:, :, ::-1], (heatmap.shape[1] // 2, heatmap.shape[0] // 2))
        heatmap = heatmap * 255

        thresholded_frame = cv2.resize(
            thresholded_frame, (thresholded_frame.shape[1] // 2, thresholded_frame.shape[0] // 2)) * 255
        thresholded_frame = cv2.cvtColor(thresholded_frame, cv2.COLOR_GRAY2BGR)
        
        upper_output_frame = np.concatenate((heatmap, thresholded_frame), axis=1)
        
        output_frame = np.concatenate(
            (upper_output_frame, lower_output_frame),
            axis=0)

        cv2.putText(output_frame, 
                    'Frame %d' % frame_num,
                    (50, 450),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    (0, 0, 255),
                    5)
        
        output_frame = output_frame.astype(np.uint8)

        output_video.write(output_frame)
        
        print('.', end='')

    input_video.release()
    output_video.release()
    cv2.destroyAllWindows()

In [19]:
HEATMAP_THRESHOLD = 4
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_TEST_VIDEO_INTPUT, 
    p_path_to_output_video=PATH_TO_TEST_VIDEO_OUTPUT,
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

......................................

In [20]:
HEATMAP_THRESHOLD = 4
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_01.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

....................................................................................................................................................................................................................

KeyboardInterrupt: 

In [ ]:
HEATMAP_THRESHOLD = 6
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_02.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 8
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_03.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 18
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_04.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 4
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_05.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 6
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_06.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 8
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_07.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 10
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_08.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 4
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64), (32, 32)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_09.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 6
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64), (32, 32)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_10.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 8
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64), (32, 32)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_11.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

In [ ]:
HEATMAP_THRESHOLD = 10
IMAGE_WINDOW_SIZES = [(128, 128), (96, 96), (64, 64), (32, 32)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_12.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)

...

/Users/tringuyen/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:39: RuntimeWarning: invalid value encountered in true_divide
/Users/tringuyen/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/matplotlib/colors.py:494: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)


...............................................................................................................

In [ ]:
HEATMAP_THRESHOLD = 14
IMAGE_WINDOW_SIZES = [(128, 128), (96, 192), (96, 96), (64, 128), (64, 64), (32, 64), (32, 32)]

process_video(
    p_path_to_input_video=PATH_TO_PROJECT_VIDEO_INTPUT, 
    p_path_to_output_video=PREFIX_PATH + 'test_video_out_14.mp4',
    p_hitmap_threshold=HEATMAP_THRESHOLD,
    p_image_window_sizes=IMAGE_WINDOW_SIZES)